# Imports and Constants

An API key must be given. Registration required first.

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2
import requests
import pandas as pd
import numpy as np

from datetime import datetime
from dateutil import parser

from dotenv import load_dotenv

from tqdm import tqdm, tqdm_notebook

# import metalhistory as mh
import metalhistory.data_query_functions as dqf

import requests
import pandas as pd
import xmltodict
import time

In [3]:
tqdm.pandas()

# For devs: Overview of functions
These functions from utils.py work (untested) so far:

In [65]:
# lastfm = dqf.LastFM()
# lastfm.get_album_info('Death', 'Symbolic', verbose=1)
# lastfm.get_album_matches('Burzum', 'Filosofem',  verbose=1)
# lastfm.get_track_info('Black Sabbath', 'Paranoid', 'War Pigs', verbose=1)

# Data Preprocessing

In [66]:
df_csv = pd.read_csv('data/MA_10k_albums.csv')
df_csv

,artist,album,MA_score
0,Slayer,Reign in Blood,36.01
1,Metallica,Kill 'Em All,33.39
2,Hades Archer,Penis Metal,32.67
3,Iron Maiden,Iron Maiden,32.38
4,Metallica,Master of Puppets,31.83
...,...,...,...
9995,Iron Maiden,Live at the Rainbow,1.92
9996,Jorn,Worldchanger,1.92
9997,Juggernaut,Trouble Within,1.92
9998,Lacrimas Profundere,Memorandum,1.92


## Collected all instances of an album

In [70]:
lastfm = dqf.LastFM()
results_df = pd.DataFrame()

LIMIT=39 # Obituary - Cause of Death is causing some error here at idx 38!

df_head = df_csv.head(LIMIT).tail(2)

for idx, row in tqdm_notebook(df_head.iterrows(), total=df_head.shape[0], desc='artists'):
    artist = row['artist']
    album = row['album']
    
    print(idx,': Querying for', artist, '-', album)
    matches = list(lastfm.get_album_matches(artist, album)['results']['albummatches'].values())[0]
    # print(matches)

    nr_matches =  len(matches)
    nr_kept_matches = 0

    for match in tqdm_notebook(matches, desc='album matches', leave=False):
        if match['artist'] == artist:
            results_df = results_df.append({
                'artist': artist,
                'album': album,
                'album_instance': match['name'],
                'lastfm_info': lastfm.get_album_info(match['artist'], match['name'], verbose=1)
            }, ignore_index=True)
            nr_kept_matches += 1        
    print('Kept', nr_kept_matches, 'out of', nr_matches, 'matches for', artist, '-', album, end='\n\n')

artists:   0%|          | 0/2 [00:00<?, ?it/s]

37 : Querying for Ghost - Opus Eponymous


album matches:   0%|          | 0/50 [00:00<?, ?it/s]

Generated API Request: http://ws.audioscrobbler.com/2.0/?&api_key=2e171c2f29c3f9b0258954e2edb289c3&method=album.getinfo&artist=Ghost&album=Opus Eponymous&format=json
Generated API Request: http://ws.audioscrobbler.com/2.0/?&api_key=2e171c2f29c3f9b0258954e2edb289c3&method=album.getinfo&artist=Ghost&album=Opus Eponymous (Japanese Edition)&format=json
Generated API Request: http://ws.audioscrobbler.com/2.0/?&api_key=2e171c2f29c3f9b0258954e2edb289c3&method=album.getinfo&artist=Ghost&album=Opus Eponymous [Bonus Track]&format=json
Generated API Request: http://ws.audioscrobbler.com/2.0/?&api_key=2e171c2f29c3f9b0258954e2edb289c3&method=album.getinfo&artist=Ghost&album=Opus eponymous [japanese edition]&format=json
Generated API Request: http://ws.audioscrobbler.com/2.0/?&api_key=2e171c2f29c3f9b0258954e2edb289c3&method=album.getinfo&artist=Ghost&album=Opus Eponymous [FL, Japan, Trooper Entertainment, XNTE-00035]&format=json
Generated API Request: http://ws.audioscrobbler.com/2.0/?&api_key=2e171

album matches:   0%|          | 0/50 [00:00<?, ?it/s]

Generated API Request: http://ws.audioscrobbler.com/2.0/?&api_key=2e171c2f29c3f9b0258954e2edb289c3&method=album.getinfo&artist=Obituary&album=Cause of Death&format=json
Generated API Request: http://ws.audioscrobbler.com/2.0/?&api_key=2e171c2f29c3f9b0258954e2edb289c3&method=album.getinfo&artist=Obituary&album=Cause of Death (Reissue)&format=json
Generated API Request: http://ws.audioscrobbler.com/2.0/?&api_key=2e171c2f29c3f9b0258954e2edb289c3&method=album.getinfo&artist=Obituary&album=Cause of Death (Remastered)&format=json
Generated API Request: http://ws.audioscrobbler.com/2.0/?&api_key=2e171c2f29c3f9b0258954e2edb289c3&method=album.getinfo&artist=Obituary&album=Cause Of Death [Bonus Tracks]&format=json
Generated API Request: http://ws.audioscrobbler.com/2.0/?&api_key=2e171c2f29c3f9b0258954e2edb289c3&method=album.getinfo&artist=Obituary&album=Cause Of Death (ORIGINAL RECORDING REMASTERED)&format=json
Generated API Request: http://ws.audioscrobbler.com/2.0/?&api_key=2e171c2f29c3f9b0258

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [15]:
results_df.head(535).tail()

,album,album_instance,artist,lastfm_info
530,...and Justice for All,...And Justice For All (1988),Metallica,"{'name': '...And Justice For All (1988)', 'art..."
531,...and Justice for All,...And Justice For All (Super Deluxe Edition),Metallica,{'name': '...And Justice For All (Super Deluxe...
532,...and Justice for All,...And Justice For All!,Metallica,"{'name': '...And Justice For All!', 'artist': ..."
533,...and Justice for All,...And Justice For All + Bonus Tracks,Metallica,NaN
534,...and Justice for All,...And Justice For All (Ultimate),Metallica,"{'name': '...And Justice For All (Ultimate)', ..."


Add some entries of the lastfm info as column to the dataframe.

In [33]:
def get_listeners(x):
    if isinstance(x, dict):
        return int(x['listeners'])
    else:
        return None

def get_playcounts(x):
    if isinstance(x, dict):
        return int(x['playcount'])
    else:
        return None


In [35]:
df = results_df
df['listeners'] = df.progress_apply(lambda row: get_listeners(row['lastfm_info']), axis=1)
df['playcount'] = df.progress_apply(lambda row: get_playcounts(row['lastfm_info']), axis=1)
df

100%|██████████| 1095/1095 [00:00<00:00, 142885.32it/s]


,album,album_instance,artist,lastfm_info,listeners,playcount
0,Reign in Blood,Reign in Blood,Slayer,"{'name': 'Reign in Blood', 'artist': 'Slayer',...",822151.0,15579798.0
1,Reign in Blood,Reign In Blood (Expanded),Slayer,"{'name': 'Reign In Blood (Expanded)', 'artist'...",165687.0,2127792.0
2,Reign in Blood,Reign In Blood (Expanded Edition),Slayer,"{'name': 'Reign In Blood (Expanded Edition)', ...",12823.0,372560.0
3,Reign in Blood,Reign In Blood (1994 Reissue),Slayer,"{'name': 'Reign In Blood (1994 Reissue)', 'art...",4856.0,130026.0
4,Reign in Blood,Reign In Blood (Remastered),Slayer,"{'name': 'Reign In Blood (Remastered)', 'artis...",2092.0,75516.0
...,...,...,...,...,...,...
1090,Wintersun,(2004) - Wintersun,Wintersun,"{'name': '(2004) - Wintersun', 'artist': 'Wint...",11.0,1383.0
1091,Wintersun,Jari Mäenpää,Wintersun,"{'name': 'Jari Mäenpää', 'artist': 'Wintersun'...",241.0,7222.0
1092,Wintersun,(2004) Wintersun,Wintersun,"{'name': '(2004) Wintersun', 'artist': 'Winter...",35.0,930.0
1093,Wintersun,Wintersun (Japan Release),Wintersun,"{'name': 'Wintersun (Japan Release)', 'artist'...",2.0,665.0


Sum up the different entries that correspond to the same album:

In [36]:
#TODO: Should probably be part of the utils.py
cumulative_df = df.drop(['album_instance'], axis=1).groupby(['artist','album']).sum()
cumulative_df

listeners    playcount
artist            album                                               
Agalloch          The Mantle                     352832.0   12132834.0
Behemoth          The Satanist                   121675.0    5022416.0
Black Sabbath     13                             194395.0    3628614.0
                  Black Sabbath                 6272170.0   78871048.0
                  Paranoid                      1937226.0   21612793.0
Burzum            Filosofem                      594290.0   13026055.0
                  Hvis lyset tar oss             224814.0    3189025.0
Children of Bodom Follow the Reaper              398938.0    9722337.0
Dark Angel        Darkness Descends               78794.0    1750565.0
Death             Human                          149273.0    3632595.0
                  Scream Bloody Gore             319957.0    7485791.0
                  Symbolic                       251175.0    6732890.0
Demolition Hammer Epidemic of Violence            39157.0     880391.0
Dissection        Storm of the Light's Bane      378476.0    9735078.0
Hades Archer      Penis Metal                       489.0       9519.0
Immortal          At the Heart of Winter         377013.0    7653635.0
Iron Maiden       Iron Maiden                   8061618.0  137694736.0
                  Killers                        376100.0    7269785.0
                  Powerslave                     764436.0   11024623.0
                  Seventh Son of a Seventh Son  1296876.0   22218933.0
                  The Number of the Beast       1071172.0   15060263.0
Judas Priest      Painkiller                     528241.0    8122534.0
Mayhem            Deathcrush                     294774.0    4724308.0
Megadeth          Countdown to Extinction        580764.0   10852391.0
                  Dystopia                        80831.0    2075582.0
                  Rust in Peace                  767270.0   16163950.0
Metallica         ...and Justice for All         940837.0   17647971.0
                  Kill 'Em All                   673071.0   16147399.0
                  Master of Puppets             1231869.0   25630035.0
                  Ride the Lightning            1163802.0   23615062.0
Morbid Angel      Altars of Madness              460416.0    9643752.0
Sepultura         Beneath the Remains            494850.0   10305247.0
Slayer            Reign in Blood                1027288.0   18550929.0
                  Show No Mercy                  264496.0    5402359.0
Sodom             Agent Orange                   100717.0    2372442.0
Wintersun         Time I                          51620.0    1831069.0
                  Wintersun                      267328.0    8785127.0

## Get tags of the most relevant album match

In [59]:
#TODO: Move these functions to the data_query_functions.py !
def get_tags(x):
    tag_names = []
    for tag in x['tags']['tag']:
        tag_names.append(tag['name'])
    return tag_names

def get_url(x):
    return x['url']

def get_info(x):
    artist = x.name[0]
    album = x.name[1]
    lastfm = dqf.LastFM()
    album_info = lastfm.get_album_info(artist, album)
    return album_info

def get_img(x):
    return x['image']

def get_mbid(x):
    try:
        return x['mbid']
    except KeyError:
        return None

def get_release_date(x):
    print(x)
    if x is not None:
        response = requests.get('http://musicbrainz.org/ws/2/release/' + str(x) + '?inc=release-groups&fmt=xml')
        while response.status_code == 503:
            print('503, wating for five seconds.')
            time.sleep(5)
            response = requests.get('http://musicbrainz.org/ws/2/release/' + str(x) + '?inc=release-groups&fmt=xml')

        response_dict = xmltodict.parse(response.text)

        return response_dict['metadata']['release']['release-group']['first-release-date']

    else:
        return None




In [60]:
cumulative_df['lastfm_info'] = cumulative_df.apply(lambda row: get_info(row), axis=1)
cumulative_df['tags'] = cumulative_df.apply(lambda row: get_tags(row['lastfm_info']), axis=1)
cumulative_df['url'] = cumulative_df.apply(lambda row: get_url(row['lastfm_info']), axis=1)
cumulative_df['images'] = cumulative_df.apply(lambda row: get_img(row['lastfm_info']), axis=1)
cumulative_df['mbid'] = cumulative_df.apply(lambda row: get_mbid(row['lastfm_info']), axis=1)
cumulative_df['release_date'] = cumulative_df.apply(lambda row: get_release_date(row['mbid']), axis=1)
cumulative_df.drop(['lastfm_info', 'url', 'images'], axis=1)


7ac99528-77a9-3624-84b7-3400f6f56e47
8f2a9448-cec0-41aa-b518-3e067c65a336
7dbf4b1f-d3e9-47bc-9194-d15b31017bd6
d4d6b8d9-413f-3aa6-9f4b-d51be1eb740c
2982b682-36ea-3605-b959-04e746736070
1ce9177c-62a0-4403-a7ee-7359026fcbf6
b6928219-0772-39ac-8156-91a609b2fd5e
6b15fc3b-ea9f-30a9-9209-c6743add0913
383aedc8-841b-40a9-b17f-00b5eb8a18dd
c5ca70aa-d86b-4a0d-84fc-910ca6011881
2ba42c9d-f654-4214-9fbb-a5d694d955d1
503, wating for five seconds.
321a3c33-9310-4b9f-b104-762e465ec60f
5b18c38f-a133-4bd4-a2f2-da6272a189df
76a7d348-f6e2-4e36-b529-b3cc003c3231
None
8c8395a0-3a02-35d6-a975-535112ec057e
25da813d-4dbd-32c0-aef0-307e790f0709
91ddcf18-98af-4f73-890c-bfc44c1d91e2
556c0066-8114-33ad-aa45-ab2f203e2777
53a1a90f-cc58-39ea-9fc2-20dd103bda01
6ea83f20-d053-3495-bc25-76aa61da13ab
7e9b0af2-5fba-4cfa-8258-23be6afe768d
08f88d68-c0bf-474a-8707-43cd58e28ab6
1a77f8a7-54ab-4568-8003-42240cd29ab0
None
2b904e74-daba-397c-a151-bafb125ceb44
4dede30f-58cf-4d43-a857-b342ad7be945
c06ed440-f25d-3127-aadb-ebe9c685b3d

listeners    playcount  \
artist            album                                                  
Agalloch          The Mantle                     352832.0   12132834.0   
Behemoth          The Satanist                   121675.0    5022416.0   
Black Sabbath     13                             194395.0    3628614.0   
                  Black Sabbath                 6272170.0   78871048.0   
                  Paranoid                      1937226.0   21612793.0   
Burzum            Filosofem                      594290.0   13026055.0   
                  Hvis lyset tar oss             224814.0    3189025.0   
Children of Bodom Follow the Reaper              398938.0    9722337.0   
Dark Angel        Darkness Descends               78794.0    1750565.0   
Death             Human                          149273.0    3632595.0   
                  Scream Bloody Gore             319957.0    7485791.0   
                  Symbolic                       251175.0    6732890.0   
Demolition Hammer Epidemic of Violence            39157.0     880391.0   
Dissection        Storm of the Light's Bane      378476.0    9735078.0   
Hades Archer      Penis Metal                       489.0       9519.0   
Immortal          At the Heart of Winter         377013.0    7653635.0   
Iron Maiden       Iron Maiden                   8061618.0  137694736.0   
                  Killers                        376100.0    7269785.0   
                  Powerslave                     764436.0   11024623.0   
                  Seventh Son of a Seventh Son  1296876.0   22218933.0   
                  The Number of the Beast       1071172.0   15060263.0   
Judas Priest      Painkiller                     528241.0    8122534.0   
Mayhem            Deathcrush                     294774.0    4724308.0   
Megadeth          Countdown to Extinction        580764.0   10852391.0   
                  Dystopia                        80831.0    2075582.0   
                  Rust in Peace                  767270.0   16163950.0   
Metallica         ...and Justice for All         940837.0   17647971.0   
                  Kill 'Em All                   673071.0   16147399.0   
                  Master of Puppets             1231869.0   25630035.0   
                  Ride the Lightning            1163802.0   23615062.0   
Morbid Angel      Altars of Madness              460416.0    9643752.0   
Sepultura         Beneath the Remains            494850.0   10305247.0   
Slayer            Reign in Blood                1027288.0   18550929.0   
                  Show No Mercy                  264496.0    5402359.0   
Sodom             Agent Orange                   100717.0    2372442.0   
Wintersun         Time I                          51620.0    1831069.0   
                  Wintersun                      267328.0    8785127.0   

                                                                                             tags  \
artist            album                                                                             
Agalloch          The Mantle                    [folk metal, doom metal, black metal, albums I...   
Behemoth          The Satanist                  [2014, Blackened Death Metal, black metal, dea...   
Black Sabbath     13                            [heavy metal, 2013, doom metal, hard rock, metal]   
                  Black Sabbath                 [heavy metal, hard rock, albums I own, 1970, c...   
                  Paranoid                      [heavy metal, hard rock, albums I own, classic...   
Burzum            Filosofem                     [black metal, ambient black metal, atmospheric...   
                  Hvis lyset tar oss            [black metal, ambient black metal, atmospheric...   
Children of Bodom Follow the Reaper             [Melodic Death Metal, albums I own, Power meta...   
Dark Angel        Darkness Descends             [thrash metal, albums I own, 1986, bay area th...   
Death             Human                         [d

In [64]:
attr_list = ['release_date', 'listeners', 'playcount', 'tags', 'mbid', 'url', 'images']
cumulative_df[attr_list].sort_values(by='listeners', ascending=False).to_csv('./data/proc_MA_10k_albums.csv')